In [1]:
import torch
import nemo.collections.asr as nemo_asr
import gc
import torchaudio
import os
import matplotlib.pyplot as plt

model = nemo_asr.models.ASRModel.from_pretrained("ai4bharat/indicconformer_stt_or_hybrid_rnnt_large")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.freeze() # inference mode
model = model.to(device) # transfer model to device


[NeMo W 2025-05-28 12:22:54 nemo_logging:349] /home/mohan.dash/miniconda3/envs/temo/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:220: FutureWarning: `torch.cuda.amp.custom_fwd(args...)` is deprecated. Please use `torch.amp.custom_fwd(args..., device_type='cuda')` instead.
      def forward(ctx, input, weight, bias, gradient_accumulation_fusion,
    
[NeMo W 2025-05-28 12:22:54 nemo_logging:349] /home/mohan.dash/miniconda3/envs/temo/lib/python3.10/site-packages/megatron/core/tensor_parallel/layers.py:250: FutureWarning: `torch.cuda.amp.custom_bwd(args...)` is deprecated. Please use `torch.amp.custom_bwd(args..., device_type='cuda')` instead.
      def backward(ctx, grad_output):
    


[NeMo I 2025-05-28 12:23:02 mixins:198] _setup_tokenizer: detected an aggregate tokenizer
[NeMo I 2025-05-28 12:23:02 mixins:335] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-05-28 12:23:02 mixins:335] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-05-28 12:23:02 mixins:335] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-05-28 12:23:02 mixins:335] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-05-28 12:23:02 mixins:335] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-05-28 12:23:02 mixins:335] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-05-28 12:23:02 mixins:335] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-05-28 12:23:02 mixins:335] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[NeMo I 2025-05-28 12:23:02 mixins:335] Tokenizer SentencePieceTokenizer initialized with 256 tokens
[

[NeMo W 2025-05-28 12:23:08 modelPT:165] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath:
    - /nlsasfs/home/ai4bharat/ai4bharat-pr/speechteam/indicasr_v3/manifests/nemo/vistaar_v3/train/train_odia.json
    sample_rate: 16000
    batch_size: 8
    shuffle: false
    num_workers: 16
    pin_memory: true
    max_duration: 30.0
    min_duration: 0.2
    is_tarred: false
    tarred_audio_filepaths: null
    shuffle_n: 2048
    bucketing_strategy: synced_randomized
    bucketing_batch_size: null
    is_concat: true
    concat_sampling_technique: temperature
    concat_sampling_temperature: 1.5
    return_language_id: true
    
[NeMo W 2025-05-28 12:23:08 modelPT:172] If you intend to do validation, please call the ModelPT.setup_validation_data() or ModelPT.setup_multiple_validation_data() method and provide a valid configuration 

[NeMo I 2025-05-28 12:23:08 features:289] PADDING: 0


[NeMo W 2025-05-28 12:23:09 nemo_logging:349] /home/mohan.dash/.local/lib/python3.10/site-packages/torch/nn/modules/rnn.py:123: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.2 and num_layers=1
      warnings.warn(
    


[NeMo I 2025-05-28 12:23:10 rnnt:1667] Vocab size for each language: 256
[NeMo I 2025-05-28 12:23:10 rnnt_models:220] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-05-28 12:23:10 rnnt_models:220] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-05-28 12:23:11 rnnt_models:220] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-05-28 12:23:11 hybrid_rnnt_ctc_bpe_models:105] Creating masks for multi-softmax layer.
[NeMo I 2025-05-28 12:23:11 rnnt_models:220] Using RNNT Loss : warprnnt_numba
    Loss warprnnt_numba_kwargs: {'fastemit_lambda': 0.0, 'clamp': -1.0}
[NeMo I 2025-05-28 12:23:12 save_restore_connector:263] Model EncDecHybridRNNTCTCBPEModel was successfully restored from /home/mohan.dash/.cache/huggingface/hub/models--ai4bharat--indicconformer_stt_or_hybrid_rnnt_large/snapsho

# Translation

In [2]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM,BitsAndBytesConfig
tokenizer = AutoTokenizer.from_pretrained("facebook/nllb-200-distilled-600M", src_lang="ory_Orya")
odia2eng_model = AutoModelForSeq2SeqLM.from_pretrained(
    "facebook/nllb-200-distilled-600M",
    # quantization_config=bnb_config,
    torch_dtype=torch.float32,
    device_map=device  # Will automatically place on GPU if available
)

In [3]:

def translate_to_english(odia_text):
    # Tokenize input
    inputs = tokenizer(odia_text, return_tensors="pt").to(model.device)

    # Generate translation in English
    translated_tokens = odia2eng_model.generate(
        **inputs, 
        forced_bos_token_id=tokenizer.convert_tokens_to_ids("eng_Latn"), 
        max_length=30
    )

    # Decode the output
    translation = tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
    return translation

In [4]:
def print_gpu_utilization():
    allocated = torch.cuda.memory_allocated() / (1024 ** 2)
    reserved = torch.cuda.memory_reserved() / (1024 ** 2)
    print(f"GPU Memory Usage>>>> Allocated: {allocated:.2f} MB |||||  Reserved:  {reserved:.2f} MB:")

def flush():
    torch.cuda.empty_cache()
    gc.collect()
    torch.cuda.empty_cache()
    gc.collect()

In [5]:
print_gpu_utilization()

GPU Memory Usage>>>> Allocated: 2870.56 MB |||||  Reserved:  5874.00 MB:


# Convert into 16kHz and Monoclannel Audio

In [14]:
!ffmpeg -i long_odia_audio.wav -ac 1 -ar 16000 infer_long_odia_audio.wav

ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --enable-libvorbis --enable-libvpx --enab

In [3]:
model.cur_decoder = "ctc" # "rnnt"
ctc_text = model.transcribe(['output_audio.wav'], batch_size=1,logprobs=False, language_id='or')[0]
print(ctc_text)
print_gpu_utilization()

Transcribing:   0%|          | 0/1 [00:00<?, ?it/s][NeMo W 2025-05-28 09:54:52 nemo_logging:349] /home/mohan.dash/t2s/NeMo/nemo/collections/asr/parts/preprocessing/features.py:417: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
      with torch.cuda.amp.autocast(enabled=False):
    
Transcribing: 100%|██████████| 1/1 [00:00<00:00,  2.43it/s]

[' ମୁଁ ଗୋଟିଏ ଥର ମୁଁ ଛିଡ଼ା ହୋଇଥିଲି ଏ ଖର ବେଳ ରାତି ବାରଟା ହୋଇଛି ମୋତେ ଜୋରରେ ପଇସା ଲାଗିଲା ଲାଇନ କଟିଯାଇଛି ଏ ଖରା ଦିନେ ହିଁ ଲାଇନ କାଟିବେ ଲାଇନବାାଲାଙ୍କର ଅଲଗା କେଉଁଠି ଲାଇନ କାଟିବେନି ସେ ଖରା ଦିନେ ହିଁ ଲାଇନ କାଟିବେ ଏମିତି ଗୋଟିଏ ଥର ଲାଇନ କଟିଦେଇଛନ୍ତି ରାତି ବାରଟା ହୋଇଛି ମୋତେ ଜୋରରେ ପରିସା ଲାଗିଲା ମୁଁ ଆସିକି ବାହାରେ ପରିସା କରୁଛି ସେମ ଟାଇମରେେ ମୁଁ ବୋଉକୁ ବି ପରିସା ଲାଗିଲା ସେ ବାହାରକୁ ପଳେଇଯାଇଛି ସଟ ଦେଖି ହ ତୋ ବୁତୋୁତୋୁ କହିଲ ବୋଉ ମୁଁ ବଢିକିଆଝ ଗଡ଼ ତତଲା ପେଜ ଢାଳକି ଘୋରା ହୋଇଛା ମୁଁ ଆଉ ଦେଖିକି ତେ ଭୁତ ହୋଇକି ଡରିବାକୁ ଚାଉୁନି ମାନେ ଏଇଭଳିଆେ ନିଜ ବୋଉ ବି ଦେଖିପାରୁନି ଅନ୍ଧାର ହେଲେ']
GPU Memory Used: 1601.81 MB
GPU Memory Total: 24564.00 MB
GPU Utilization: 2%


# Long Audio

In [6]:
audio_path='infer_long_odia_audio.wav'
sample_rate = 16000
waveform, sr = torchaudio.load(audio_path)
assert sr == sample_rate, f"Sample rate must be {sample_rate}, got {sr}"

In [7]:
chunk_duration_sec=10
overlap_sec=0

total_samples = waveform.shape[1]
chunk_samples = int(chunk_duration_sec * sample_rate)
overlap_samples = int(overlap_sec * sample_rate)

In [8]:
total_samples,chunk_samples,overlap_samples

(11595117, 160000, 0)

In [ ]:
from tqdm.auto import tqdm
import shutil

os.makedirs('temp_audio',exist_ok=True)

starts = list(range(0, total_samples, chunk_samples - overlap_samples))

full_transcription=[]
english_translation=[]

for idx,start in tqdm(enumerate(starts),total=len(starts)):
    chunk = waveform[:, start:start + chunk_samples]
    torchaudio.save(f'temp_audio/{idx}.wav', chunk, sample_rate)
    rnnt_text = model.transcribe([f'temp_audio/{idx}.wav'], batch_size=1, language_id='or',verbose=False )[0][0]
    english_translation.append(translate_to_english(rnnt_text))
    full_transcription.append(rnnt_text)
    
shutil.rmtree('temp_audio')

  0%|          | 0/73 [00:00<?, ?it/s]

In [12]:
full_transcription

[' ନମସ୍କାର ଆଜି ହେଉଛି ମେ ଅଠାଇଶି ତାରିଖ ବୁଧବାର ଆପଣମାନଙ୍କ ପାଇଁ ନେଇକି ଆସିଛି ଆଜି ଦିନର କେତୋଟି ଗୁରୁତ୍ୱପୂର୍ଣ୍ଣ ଖବର ସେଥିପାଇଁ ଭିଡିଓକୁ ଶେଷ ପର୍ଯ୍ୟନ୍ତ ଦେଖନ୍ତୁ ଏବଂ ଆପଣଙ୍କ ଭଲପାଇବା ସ୍ୱରୂପ',
 ' ଗୋଟେ ଗୋଟେ ଲିକ କରିଦିଅନ୍ତୁ ଏବଂ ଚନେଲ୍ କୁ ଯଦି ନୂଆ ଆସିଛନ୍ତି ତାହେଲେ ଚନେଲ କୁ ସୁବ୍ସ୍କ୍ରିବ କରି ପାଖରେ ଆସୁଥିବା ବେଲଇଟନ କୁ କ୍ଲିକ କରି ରଖନ୍ତୁ ଚାଲନ୍ତୁ ଆଜି ଦିନର ଗୁରୁତ୍୍ୱପୂର୍ଣ୍ଣ ଖବର ଉପରେ ନଜର ପକେଇବା ପ୍ରଥମ',
 ' ଖବର ଆବାସ ଯୋଜନାକୁ ନେଇ ଆସିଲା ବଡ଼ ଖବର ମେ ଏକତିରିଶି ତାରିଖ ହେଉଛି ପ୍ରଧାନମନ୍ତ୍ରୀ ଆବାସ ଯୋଜନାରେ ଆବେଦନ କରିବାର ଶେଷ ତାରିଖ ତୁରନ୍ତ କରନ୍ତୁ ଆବେଦନ ଏନେଇ ପଞ୍ଚାୟତିରାଜ ମନ୍ତ୍ରୀ ରବୀନ',
 ' ରବ ନାରାୟଣ ନାୟକ ସୂଚନା ଦେଇଛନ୍ତି ବର୍ତ୍ତମାନ ସୁଦ୍ଧା ଛତିଶି ଲକ୍ଷ ଏକାଅଶୀ ହଜାର ତାଲିକା ପହଞ୍ଚିଛି ଯେଉଁମାନେ ପଞ୍ଜୀକରଣ କରିନାହାନ୍ତି ତୁରନ୍ତ କର ବୋଲି ପଞ୍ଚାୟତିରାଜ ମନ୍ତ୍ରୀ ରବିିନାରାୟଣ ନାୟକ କହିଛନ୍ତି',
 ' ତେବେ ମାସେ କି ଦୁଇ ମାସ ଭିତରେ ହିତାଧିକାରୀଙ୍କ ଚୟନ ସରିବ କେନ୍ଦ୍ର ସରକାରଙ୍କ ଦ୍ୱାରା ଲଞ୍ଚ କରାଯାଇଥିବା ମୋବାଇଲ ଆପ୍ ଆବାସ ପ୍ଲସ ଦୁଇ ହଜାର ଚବିଶି ଏହି ଆପ୍ ବ୍ୟବହାର କରି ଆପଣ ଏବେ ପ୍ରଧାନମନ୍ତ୍ରୀ',
 ' ଆବାସ ଯୋଜନା ଅଧୀନରେ ଆପଣଙ୍କ ଘର ପାଇଁ ସହଜରେ ଆବେଦନ କରିପାରିବେ ପ୍ରଧାନମନ୍ତ୍ରୀ ଆବାସ ଯୋଜନାରେ ମେ ଏକତିରିଶ ତାରିଖ ପର୍ଯ୍ୟନ୍ତ ଆବେଦନ 

In [13]:
english_translation

['Today is the 28th of May, Wednesday I bring you some important news of the day so watch the video till the end and your love',
 "Click on the next ballpoint to subscribe to the channel and if you are new to the channel, let's take a look at",
 'The last date to apply for PM Awas Yojana is May 31.',
 'Rav Narayan Naik has informed that so far, there have been 631 lakh lists of people who have not registered,',
 'However, the selection of beneficiaries will be completed within a month or two. The mobile app Abhaas Plus launched by the Union',
 'You can easily apply for your home under Awas Yojana Prime Minister Awas Yojana by May 31st after which the application will not',
 'The Minister also said that the government has plans to get the houses after monsoon.',
 'If you are excluded from these schemes, you will get houses under the Endodaya Yojana, said Ravinarayan Na',
 '60 thousand houses have been given now another lakh houses are reported to be given next news in the Bay of Bengal'

In [23]:
english_translation

['Today is the 28th of May, Wednesday I bring you some important news of the day so watch the video till the end and your love',
 "Click on the next balllight to subscribe to the channel and if you are new to the channel, let's first look at the",
 'The last date to apply for PM Awas Yojana is May 31.',
 'Rav Narayan Naik has informed that so far, there have been 631 lakh lists of people who have not registered,',
 'The mobile app launched by the Union Government, Abhaas Plus, is available in a month or two.',
 'You can easily apply for your home under Awas Yojana Prime Minister Awas Yojana by May 31st after which the application will not',
 'The Minister also said that the government has plans to get the houses after monsoon.',
 'If you are excluded from these schemes, you will get houses under the Endodaya Yojana, said Ravinarayan Na',
 '60 thousand houses have been given now another lakh houses are reported to be given next news in the Bay of Bengal',
 'The Special Relief Commission